# **Conversational Chatbot for NESTLE HR using Gradio**

In [1]:
#Import necessary libraries
import os
import openai
import sys
import gradio as gr
import time

In [2]:
#Using PyPDF
from langchain.document_loaders import PyPDFLoader

Doc_loader = PyPDFLoader("NESTLE_HR_POLICY_2012.pdf")
extracted_text = Doc_loader.load()

In [3]:
# Text splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter  = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
splitted_text=text_splitter.split_documents(extracted_text)

In [4]:
# Embeddings
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

/tmp/ipykernel_225/2304085244.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [5]:
# Vector store
from langchain.vectorstores import FAISS

In [6]:
vectordb = FAISS.from_documents(
    documents=splitted_text,
    embedding=embeddings
)

In [7]:
# LLM
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

/tmp/ipykernel_225/3107502072.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [8]:
# Retrieval chain
from langchain.chains import RetrievalQA
Retriever_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       )

In [9]:
# Function to handle chatbot queries
def chatbot_response(message, history):
    """
    Function to process user queries and return responses from the HR chatbot
    """
    if message.strip() == "":
        return "Please enter a valid question about Nestle's HR policies."
    
    try:
        # Get the answer from the chain
        start = time.time()
        res = Retriever_chain(message)
        end = time.time()
        
        # Format the response
        response = res['result']
        processing_time = f"\n\n⏱️ *Response generated in {round(end - start, 2)} seconds*"
        
        return response + processing_time
        
    except Exception as e:
        return f"Sorry, I encountered an error while processing your question: {str(e)}"

In [10]:
# Custom CSS for styling
custom_css = """
#title {
    text-align: center;
    background: linear-gradient(45deg, #FF6B6B, #4ECDC4);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-size: 2.5em;
    font-weight: bold;
    margin-bottom: 20px;
}

#description {
    text-align: center;
    font-size: 1.2em;
    color: #666;
    margin-bottom: 30px;
}

.gradio-container {
    max-width: 800px;
    margin: 0 auto;
}
"""

In [11]:
# Create Gradio interface
with gr.Blocks(css=custom_css, title="Nestle HR Assistant") as demo:
    
    # Title and description
    gr.HTML("""
        <div id="title">🏢 Nestle HR Policy Assistant</div>
        <div id="description">
            Ask me anything about Nestle's HR policies and procedures. 
            I'm here to help you find the information you need quickly and accurately.
        </div>
    """)
    
    # Create chatbot interface
    chatbot = gr.Chatbot(
        value=[],
        height=400,
        bubble_full_width=False,
        show_label=False,
        container=True
    )
    
    # Create message input
    msg = gr.Textbox(
        placeholder="Ask me about Nestle's HR policies (e.g., 'What is the leave policy?', 'How do I apply for maternity leave?')",
        show_label=False,
        max_lines=3,
        container=False
    )
    
    # Create buttons
    with gr.Row():
        submit_btn = gr.Button("Send 📤", variant="primary", scale=1)
        clear_btn = gr.Button("Clear Chat 🗑️", scale=1)
    
    # Example questions
    gr.Examples(
        examples=[
            ["What is Nestle's vacation policy?"],
            ["How do I request time off?"],
            ["What are the working hours?"],
            ["Tell me about employee benefits"],
            ["What is the dress code policy?"],
            ["How do I report workplace issues?"]
        ],
        inputs=msg,
        label="💡 Try these example questions:"
    )
    
    # Function to handle user input
    def user_input(message, history):
        return "", history + [[message, None]]
    
    def bot_response(history):
        user_message = history[-1][0]
        bot_message = chatbot_response(user_message, history)
        history[-1][1] = bot_message
        return history
    
    # Set up event handlers
    msg.submit(user_input, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot_response, chatbot, chatbot
    )
    
    submit_btn.click(user_input, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot_response, chatbot, chatbot
    )
    
    clear_btn.click(lambda: ([], ""), outputs=[chatbot, msg])
    
    # Footer
    gr.HTML("""
        <div style="text-align: center; margin-top: 20px; color: #888; font-size: 0.9em;">
            <p>🤖 Powered by OpenAI GPT-3.5 Turbo & LangChain | Built with ❤️ using Gradio for AGS Advances Generative AI: Building Applications</p>
            <p>📋 This assistant is trained on Nestle's HR Policy Document (2012)</p>
        </div>
    """)

/tmp/ipykernel_225/2022210204.py:14: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipykernel_225/2022210204.py:14: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


In [ ]:
# Launch the interface
if __name__ == "__main__":
    demo.launch(
        share=True,          # Creates a public link
        debug=True,          # Enable debug mode
        show_error=True,     # Show errors in the interface
        server_name="0.0.0.0",  # Allow external access
        server_port=7860     # Default Gradio port
    )

* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://d2e929e1611e806ab8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipykernel_225/3825980550.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = Retriever_chain(message)
